# 1. Introduction

# 1.1. Overview of the extension

The ndx-icephys-meta extension at https://github.com/oruebel/ndx-icephys-meta defines 5 new main tables for organizing ICEphys metadata:

* IntracellularRecordings,
* Sweeps,
* SweepSequences,
* Runs,
* Conditions

To make these more easily accesible and to allow us to store the tables in ``/general/intracellular__ephys`` in the NWBFile, the extension also extends NWBFile itself via the new type:
* ICEphysFile

ICEphysFile makes all 5 tables accesible via corresponding properties, manages creation of the tables, and provides convenienc functions for populating the tables. In addition, ICEphysFile also declares SweepTable as deprecated in the schema and adds warnings if SweepTable is being used.

**Note:** Upon merging of this proposal with the NWB core, the ``ICEphysFile`` neurodata_type would be removed and NWBFile updated accordingly instead. 

# 1.2: Problem:

For storage and queries on specific tables it is useful to represent data in a normalized fashion in a hierarchy of tables as described in above. This avoids repeating data values (which can easily lead to errors and data inconsitencies) and allows us to interact with the data at the approbriate level. 

For visualization and queries across tables, however, it can be useful to represent the data in a single table (i.e., Pandas DataFrame). 

# 1.3: Approach:

To address this problem we provide the convience functions``to_hierarchical_dataframe()`` and ``to_denormalized_dataframe()``, which allow us to convert the hierarchy of tables into a single, consolidated table (i.e. Pandas DataFrame). These functions are available on all hierarchcial tables, i.e.,  Sweeps, SweepSequences, Runs, and Conditions (but not IntracellularRecordings, since it does not reference other tables, so we can just use the regular ``to_dataframe()`` function). With this, we can use Pandas advanced query functionality to search our table. 

# 1.4: Imports and Requirement

**Requirement** Implementation of this functionality requires https://github.com/hdmf-dev/hdmf/pull/191 for HDMF

In [1]:
# Standard Python imports
from datetime import datetime
from dateutil.tz import tzlocal
import numpy as np
import pandas as pd

In [2]:
# Standard PyNWB imports
from pynwb.icephys import VoltageClampStimulusSeries, VoltageClampSeries
from pynwb import NWBHDF5IO

In [3]:
# Imports needed from the ndx-icephys-meta
from ndx_icephys_meta.icephys import ICEphysFile

Update CSS styling to show cell borders when displaying Pandas DataFrame objects in the notebook.

In [4]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# 2. Create a basic NWB File for testing

**The following parts of the code are for basic setup only and are unchanged from what is in the current NWB release**

In [5]:
# Create the file
nwbfile = ICEphysFile(
            session_description='my first synthetic recording',
            identifier='EXAMPLE_ID',
            session_start_time=datetime.now(tzlocal()),
            experimenter='Dr. Bilbo Baggins',
            lab='Bag End Laboratory',
            institution='University of Middle Earth at the Shire',
            experiment_description='I went on an adventure with thirteen dwarves to reclaim vast treasures.',
            session_id='LONELYMTN')
# Add a device
device = nwbfile.create_device(name='Heka ITC-1600')
# Add an intracellular electrode
electrode = nwbfile.create_icephys_electrode(name="elec0",
                                        description='a mock intracellular electrode',
                                        device=device)

def create_stimulus_and_response(sweep_number, electrode):
    """
    Helper function to construct a dummy stimulus and reponse pair representing an 
    instracellular recording:
    
    Args: 
       - sweep_number : Integer sweep number of the recording
       - electrode: Intracellular electrode used 
       
    Returns: Tuple of CurrentClampStimulusSeries with the stimulus and 
             VoltageClampSeries with the response.
    """
    stimulus = VoltageClampStimulusSeries(
                name="ccss",
                data=[1, 2, 3, 4, 5],
                starting_time=123.6,
                rate=10e3,
                electrode=electrode,
                gain=0.02,
                sweep_number=sweep_number)
    # Create and ic-response
    response = VoltageClampSeries(
                name='vcs',
                data=[0.1, 0.2, 0.3, 0.4, 0.5],
                conversion=1e-12,
                resolution=np.nan,
                starting_time=123.6,
                rate=20e3,
                electrode=electrode,
                gain=0.02,
                capacitance_slow=100e-12,
                resistance_comp_correction=70.0,
                sweep_number=sweep_number)
    return stimulus, response

# 3. Construct our Intracellular Electrophysiolgy Metadata tables

**The parts in this section are what consitutes the new elements from this proposal.** 

**Note:** To illustrate the behavior with custom metadata, we add some arbitrary metadata columns to each table. 
**Note:** We construct at least 2 rows in each table to have at least a minimal representative set of rows, rather than just having 1 row per table which does not lend itself well to illustrate the problem we are trying to address here.

### Add intracelluar recordings
Add a set of intracellular recordings, each consisting of and electrode, stimulus, and reponse. 

In [6]:
# Add some intracellular recordings as fake data to construct an illustrative hierarchy
for sweep_number in range(10):
    stim, resp = create_stimulus_and_response(sweep_number, electrode)
    nwbfile.add_intracellular_recording(electrode=electrode,
                                        stimulus=stim,
                                        response=resp,
                                        id=sweep_number # optional
                                       )
nwbfile.intracellular_recordings.add_column(name='stimulus_type', data=['A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'C3', 'D1', 'D2', 'D3'], 
                                            description='String indicating the type of stimulus applied')

### Add a set of sweep
Add a set of sweeps, each consisting of a set of intracellular recordings. The ``recordings`` argument of the ``add_sweep`` function here is simply a list of ints with the indices of the corresponding rows in the IntracellularRecordings table. We use the same strategy also for the other tables, where we indicate the rows we reference based on their index. 

**NOTE:** In practice each recording should belong to a single sweep. For illustration purposes and to test the code we here add the row with index 8 from our intracellular recordings table to both the first and last sweep. 

In [7]:
nwbfile.add_icephys_sweep(recordings=[0,1,8], id=10)  
nwbfile.add_icephys_sweep(recordings=[2,3], id=11)
nwbfile.add_icephys_sweep(recordings=[4,5,6], id=12)
nwbfile.add_icephys_sweep(recordings=[7,8,9], id=13)
nwbfile.icephys_sweeps.add_column(name='tag', data=np.arange(4), description='integer tag for a sweep')

### Add sweep sequences
Add a set of sweep sequence consisting of a set of sweeps. 

In [8]:
nwbfile.add_icephys_sweep_sequence(sweeps=[0,1], id=20)
nwbfile.add_icephys_sweep_sequence(sweeps=[2], id=21)
nwbfile.add_icephys_sweep_sequence(sweeps=[3], id=22)
nwbfile.icephys_sweep_sequences.add_column(name='type', data=['T2', 'T1.1', 'T1.2'], description='type of the sweep sequence')

### Add runs
Add a set of runs, each consisting of a set of sweep sequences

In [9]:
nwbfile.add_icephys_run(sweep_sequences=[0], id=30)
nwbfile.add_icephys_run(sweep_sequences=[1,2], id=31)
nwbfile.icephys_runs.add_column(name='number', data=np.arange(2), description='integer number of our run')

### Add conditions
Add a set of conditions, each consisting of a set of runs

In [10]:
nwbfile.add_icephys_condition(runs=[0], id=40)
nwbfile.add_icephys_condition(runs=[1], id=41)
nwbfile.icephys_conditions.add_column(name='temperature', data=[32., 24.], description='Temperatur in C')

# 4. Visualize the individual data tables

Using the ``to_dataframe`` function provided by the standard ``DynamicTable`` class in HDMF we can visualize and query the indiviudal tables.

In [11]:
print(nwbfile.intracellular_recordings.name)
display(nwbfile.intracellular_recordings.to_dataframe())
print(nwbfile.icephys_sweeps.name)
display(nwbfile.icephys_sweeps.to_dataframe())
print(nwbfile.icephys_sweep_sequences.name)
display(nwbfile.icephys_sweep_sequences.to_dataframe())
print(nwbfile.icephys_runs.name)
display(nwbfile.icephys_runs.to_dataframe())
print(nwbfile.icephys_conditions.name)
display(nwbfile.icephys_conditions.to_dataframe())

intracellular_recordings


,stimulus,response,electrode,stimulus_type
id,,,,
0,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A1
1,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A2
2,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,B1
3,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,B2
4,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C1
5,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C2
6,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C3
7,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D1
8,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D2


sweeps


,recordings,tag
id,,
10,s...,0
11,s...,1
12,s...,2
13,s...,3


sweep_sequences


,sweeps,type
id,,
20,rec...,T2
21,rec...,T1.1
22,rec...,T1.2


runs


,sweep_sequences,number
id,,
30,...,0
31,...,1


conditions


,runs,temperature
id,,
40,sweep_se...,32.0
41,sweep_se...,24.0


**Note** Here the columns that reference other tables store itself Pandas DataFrames as values. E.g., the ``recordings`` column of the ``Sweeps`` table will contain in each row a DataFrame describing the relevant rows from the ``IntracellularRecordings`` table. As an example let's have a look at our first sweep. Recall, that our first sweep refernced the rows``[0,1,8]`` in our ``IntracellularRecordings`` table. As expected the below then shows a Pandas DataFrame with the data from those three rows.

In [12]:
sweeps_df = nwbfile.icephys_sweeps.to_dataframe()  # convert the sweeps table to Pandas
first_sweep = sweeps_df.iloc[0]               # select the first sweep (i.e, row)
first_sweep_recordings = first_sweep.loc['recordings']  # Get the recordings from the first sweep
# The short version for the above would simply be: 
# nwbfile.icephys_sweeps.to_dataframe().iloc[0].loc['recordings']

In [13]:
display(first_sweep_recordings)               # Display the results

,stimulus,response,electrode,stimulus_type
id,,,,
0,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A1
1,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A2
8,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D2


# 5. Construct a DataFrame with a hierarchcial MultiIndex on rows 

A hierarchical dataframe is pandas DataFrame with a MultiIndex describing row indices at different levels. This allows us to visualize all data in a single dataframe. The columns of the dataframe represent the columns of the table at the bottom of our hierarchy (i.e., IntracellularRecordings) and the MultiIndex contains the columns from all other tables in the hierarchy. 

As a first example, let's look at the Sweeps table (i.e., the first table in our hierarchy). As we can see, the DataFrame table shows the stimulus, response, electrode, and custom stimulus_type from our IntracellularRecordings table as columns. Since we included intracellular recording with id=8 in two separate sweeps it also appears twice here here (once in the 3rd and 10th row). The columns of the sweeps tables (i.e., id and tag) and the index from the intracellular recordings (i.e., id) appear as part of the MultiIndex of the table, at level 1 and 2, respectively. 

In [14]:
print(nwbfile.icephys_sweeps.name)
nwbfile.icephys_sweeps.to_hierarchical_dataframe()

sweeps


source_table         intracellular_recordings  \
label                                      id   
sweeps_id sweeps_tag                            
10        0                                 0   
          0                                 1   
          0                                 8   
11        1                                 2   
          1                                 3   
12        2                                 4   
          2                                 5   
          2                                 6   
13        3                                 7   
          3                                 8   
          3                                 9   

source_table                                                             \
label                                                          stimulus   
sweeps_id sweeps_tag                                                      
10        0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
11        1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
12        2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
13        3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
          3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   

source_table                                                             \
label                                                          response   
sweeps_id sweeps_tag                                                      
10        0           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          0           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          0           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
11        1           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          1           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
12        2           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          2           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          2           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
13        3           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          3           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
          3           (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   

source_table                                                             \
label                                                         electrode   
sweeps_id sweeps_tag                                                      
10        0           elec0 pynwb.icephys.IntracellularElectrode at ...   
          0           elec0 pynwb.icephys.IntracellularElectrode at ...   
          0           elec0 pynwb.icephys.IntracellularElectrode at ...   
11        1           elec0 pynwb.icephys.IntracellularElectrode at ...   
          1           elec0 pynwb.icephys.IntracellularElectrode at ...   
12        2           elec0 pynwb.icephys.IntracellularElectrode at ...   
          2           elec0 pynwb.icephys.IntracellularElectrode at ...   
          2           elec0 pynwb.icephys.IntracellularElectrode at ...   
13        3           elec0 pynwb.icephys.IntracellularElectrode at ...   
          3           elec0 pynwb.icephys.IntracellularElectrode at ...   
          3           elec0 pynwb.icephys.IntracellularElectrode at ...   

source_table                        
label                stimulus_type  
sweeps_id sweeps_tag                
10        0                     A1  
          0 

Next, let us do the same for our top-most conditions table. Here the conversion to a hiearchical dataframe recursively resolves all levels of the hierarchy, so that all data from our intracellular ephys metadata tables is represented in a single hierarchcial table.

In [15]:
print(nwbfile.icephys_conditions.name)
nwbfile.icephys_conditions.to_hierarchical_dataframe()

conditions


source_table                                                                                                          intracellular_recordings  \
label                                                                                                                                       id   
conditions_id conditions_temperature runs_id runs_number sweep_sequences_id sweep_sequences_type sweeps_id sweeps_tag                            
40            32.0                   30      0           20                 T2                   10        0                                 0   
                                                                                                           0                                 1   
                                                                                                           0                                 8   
                                                                                                 11        1                                 2   
                                                                                                           1                                 3   
41            24.0                   31      1           21                 T1.1                 12        2                                 4   
                                                                                                           2                                 5   
                                                                                                           2                                 6   
                                                         22                 T1.2                 13        3                                 7   
                                                                                                           3                                 8   
                                                                                                           3                                 9   

source_table                                                                                                                                                              \
label                                                                                                                                                           stimulus   
conditions_id conditions_temperature runs_id runs_number sweep_sequences_id sweep_sequences_type sweeps_id sweeps_tag                                                      
40            32.0                   30      0           20                 T2                   10        0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                 11        1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
41            24.0                   31      1           21                 T1.1                 12        2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                         22    

**NOTE** If we only want the hierarichal multiindex on the rows but not the MultiIndex on the columns then we can simply the ``flat_column_index=True`` when calling the ``to_hierarchical_dataframe()`` functions. For example:

In [16]:
print(nwbfile.icephys_sweeps.name)
#nwbfile.icephys_sweeps.to_hierarchical_dataframe(flat_column_index=True)
nwbfile.icephys_conditions.to_hierarchical_dataframe(flat_column_index=True)

sweeps


id  \
conditions_id conditions_temperature runs_id runs_number sweep_sequences_id sweep_sequences_type sweeps_id sweeps_tag       
40            32.0                   30      0           20                 T2                   10        0            0   
                                                                                                           0            1   
                                                                                                           0            8   
                                                                                                 11        1            2   
                                                                                                           1            3   
41            24.0                   31      1           21                 T1.1                 12        2            4   
                                                                                                           2            5   
                                                                                                           2            6   
                                                         22                 T1.2                 13        3            7   
                                                                                                           3            8   
                                                                                                           3            9   

                                                                                                                                                                stimulus  \
conditions_id conditions_temperature runs_id runs_number sweep_sequences_id sweep_sequences_type sweeps_id sweeps_tag                                                      
40            32.0                   30      0           20                 T2                   10        0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           0           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                 11        1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           1           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
41            24.0                   31      1           21                 T1.1                 12        2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           2           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                         22                 T1.2                 13        3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
                                                                                                           3           (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   

                                                                                                                                                                response  \
conditions_id conditions_temperature runs_id runs_number sweep_sequences_id sweep_se

# 6. Construct a denormalized dataframe with all data expanded into columns

For some use-cases it may be useful to represent all data in a single, denormalized table, i.e., a table where all data is represented in columns where redundant values from the hierarchy are repeated in each column as necessary. We can do this using the ``to_denormalized_dataframe()``.  Here we store the lable of each columns and name of the source table each column comes from as a MultiIndex so that columns are grouped by the table they came from.

In [17]:
print(nwbfile.icephys_conditions.name)
nwbfile.icephys_conditions.to_denormalized_dataframe()

conditions


source_table conditions             runs        sweep_sequences       sweeps  \
label                id temperature   id number              id  type     id   
0                    40        32.0   30      0              20    T2     10   
1                    40        32.0   30      0              20    T2     10   
2                    40        32.0   30      0              20    T2     10   
3                    40        32.0   30      0              20    T2     11   
4                    40        32.0   30      0              20    T2     11   
5                    41        24.0   31      1              21  T1.1     12   
6                    41        24.0   31      1              21  T1.1     12   
7                    41        24.0   31      1              21  T1.1     12   
8                    41        24.0   31      1              22  T1.2     13   
9                    41        24.0   31      1              22  T1.2     13   
10                   41        24.0   31      1              22  T1.2     13   

source_table     intracellular_recordings  \
label        tag                       id   
0              0                        0   
1              0                        1   
2              0                        8   
3              1                        2   
4              1                        3   
5              2                        4   
6              2                        5   
7              2                        6   
8              3                        7   
9              3                        8   
10             3                        9   

source_table                                                     \
label                                                  stimulus   
0             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
1             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
2             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
3             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
4             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
5             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
6             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
7             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
8             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
9             (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   
10            (0, 5, ccss pynwb.icephys.VoltageClampStimulus...   

source_table                                                     \
label                                                  response   
0             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
1             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
2             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
3             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
4             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
5             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
6             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
7             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
8             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
9             (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   
10            (0, 5, vcs pynwb.icephys.VoltageClampSeries at...   

source_table                                                                   
label                                                 electrode stimulus_type  
0             elec0 pynwb.icephys.IntracellularElectrode at ...            A1  
1             elec0 pynwb.icephys.IntracellularElectrode at ...            A2  
2             elec0 pynwb.icephys.IntracellularElectrode at ...            D2  
3             elec0 pynwb.icephys.IntracellularElectrode at ...            B1  
4             elec0 pynwb.icephys.IntracellularElectrode at ...            B2  
5             elec0 pynwb.icephys.IntracellularElectrode at ...      

**NOTE** Similar to the ``to_hierarchical_dataframe()``,  ff we only do not want the MultiIndex on the columns then we can simply set ``flat_column_index=True`` when calling the ``to_denormalized_dataframe`` functions. For example:

In [18]:
print(nwbfile.icephys_conditions.name)
nwbfile.icephys_conditions.to_denormalized_dataframe(flat_column_index=True)

conditions


,conditions_id,conditions_temperature,runs_id,runs_number,sweep_sequences_id,sweep_sequences_type,sweeps_id,sweeps_tag,id,stimulus,response,electrode,stimulus_type
0,40,32.0,30,0,20,T2,10,0,0,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A1
1,40,32.0,30,0,20,T2,10,0,1,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,A2
2,40,32.0,30,0,20,T2,10,0,8,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D2
3,40,32.0,30,0,20,T2,11,1,2,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,B1
4,40,32.0,30,0,20,T2,11,1,3,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,B2
5,41,24.0,31,1,21,T1.1,12,2,4,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C1
6,41,24.0,31,1,21,T1.1,12,2,5,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C2
7,41,24.0,31,1,21,T1.1,12,2,6,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,C3
8,41,24.0,31,1,22,T1.2,13,3,7,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D1
9,41,24.0,31,1,22,T1.2,13,3,8,"(0, 5, ccss pynwb.icephys.VoltageClampStimulus...","(0, 5, vcs pynwb.icephys.VoltageClampSeries at...",elec0 pynwb.icephys.IntracellularElectrode at ...,D2


# 7. Export

Once we have the data in Pandas we can easily export all our metadata to other formats, e.g., Excel, LaTeX, CSV, among many others:

In [19]:
try:
    nwbfile.icephys_conditions.to_denormalized_dataframe().to_excel('icephys_meta.xlsx', engine='openpyxl')
except ModuleNotFoundError as e:
    print(e)

In [20]:
print(nwbfile.icephys_conditions.to_denormalized_dataframe().to_latex())

\begin{tabular}{lrrrrrlrrrllll}
\toprule
source\_table & \multicolumn{2}{l}{conditions} & \multicolumn{2}{l}{runs} & \multicolumn{2}{l}{sweep\_sequences} & \multicolumn{2}{l}{sweeps} & \multicolumn{5}{l}{intracellular\_recordings} \\
label &         id & temperature &   id & number &              id &  type &     id & tag &                       id &                                           stimulus &                                           response &                                          electrode & stimulus\_type \\
\midrule
0  &         40 &        32.0 &   30 &      0 &              20 &    T2 &     10 &   0 &                        0 &  (0, 5, ccss pynwb.icephys.VoltageClampStimulus... &  (0, 5, vcs pynwb.icephys.VoltageClampSeries at... &  elec0 pynwb.icephys.IntracellularElectrode at ... &            A1 \\
1  &         40 &        32.0 &   30 &      0 &              20 &    T2 &     10 &   0 &                        1 &  (0, 5, ccss pynwb.icephys.VoltageClampStimulus... & 